In [1]:
import requests
from bs4 import BeautifulSoup, SoupStrainer
import pandas as pd
import re
import json
import pickle

In [2]:
page = requests.get('https://www.newtral.es/zona-verificacion/fakes/')

In [3]:
#print ("pagina", page)
print ("pagina status code", page.status_code)
#print ("contenido de la pagina", page.content)

pagina status code 200


In [4]:
soup = BeautifulSoup(page.content, 'html.parser')
#print(soup)
#print(soup.prettify())

In [5]:
def get_urls(noticias):
    
    page = requests.get('https://www.newtral.es/wp-json/wp/v2/posts?per_page='+ str(noticias) +'&offset=0&categories=59&_locale=user')

    for e in page.json():
        enlace = e['link']
        lista_enlaces.append(enlace)
        
    return lista_enlaces

In [6]:
def scraping_article(url):
    
    #article = {"Title":"Titulo_noticia", "Publication_date":"Fecha_noticia", "Content":"Article_content"}

    page = requests.get(url)

    soup3 = BeautifulSoup(page.content, 'html.parser')

    title = soup3.find("h1",class_="entry-title c-article__title").get_text()
    #print(title)
    
    date = soup3.find("time",class_="entry-date c-article__date published updated").get_text().strip('\n')
    #print(date)
    
    content_list = []
    body = soup3.find("div", class_="entry-content")
    body2 = body.find_all("p", class_="")
    
    for e in body2:
        content_list.append(e.get_text().replace("\xa0", " "))
    
    content = ''.join(content_list[:-1])
    
    article = {"Title": title, "Publication_date": date, "Content": content, "URL": url}
    
    return article

In [7]:
noticias = 37 #En esta web tenemos 13 noticias de portada y luego se van sumando de 12 en 12 (Podemos seleccionar un número intermedio)
lista_enlaces = []

lista_enlaces = get_urls(noticias)

#print(len(lista_enlaces))
lista_enlaces

lista_articulos = []   
for e in lista_enlaces:
    articulo = scraping_article(e)
    lista_articulos.append(articulo)


In [8]:
print(len(lista_enlaces))
print(lista_enlaces)

37
['https://www.newtral.es/bulo-video-helena-resano-disculpas-negro-la-sexta/20200528/', 'https://www.newtral.es/la-fachada-de-la-fundacion-gates-no-dice-centro-para-la-reduccion-global-de-la-poblacion-humana/20200527/', 'https://www.newtral.es/montero-recorte-sueldos-populismo/20200527/', 'https://www.newtral.es/bulo-podemos-quitar-voto-personas-mayores/20200527/', 'https://www.newtral.es/bulo-portada-bolsonaro-folha/20200526/', 'https://www.newtral.es/no-van-a-habilitar-un-pabellon-en-fraga-huesca-por-un-repunte-de-casos-de-covid-19-es-para-acoger-a-temporeros-sin-hogar/20200525/', 'https://www.newtral.es/es-falso-que-el-video-de-una-ambulancia-que-atraviesa-la-caravana-de-vox-en-santander-sea-de-2017/20200524/', 'https://www.newtral.es/el-coronavirus-no-se-multiplica-en-aguas-residuales-y-echar-lejia-por-las-canalizaciones-afectaria-a-las-investigaciones-cientificas/20200524/', 'https://www.newtral.es/documentos-que-parecen-oficiales-pero-estan-manipulados-ii/20200523/', 'https://w

In [9]:
print(lista_articulos)

[{'Title': 'El video de Helena Resano pidiendo disculpas tras emitirse un fragmento de entrevista con el presidente de SOS Racismo está manipulado', 'Publication_date': '28 mayo 2020', 'Content': 'Nos habéis preguntado por este supuesto vídeo de laSexta Noticias en el que se ve una entrevista a Moha Gerehou, presidente de SOS Racismo, que se corta repentinamente y se va a un plano de la periodista Helena Resano en el que dice: “Pedimos disculpas por ese negro que nos ha entrado en ese vídeo en la parte final de esa información”. Varias cuentas de Twitter lo han compartido pero es un FAKE, se trata de un vídeo manipulado.El informativo de Helena Resano es del 9 diciembre de 2015 y se ve en el fondo una pantalla donde se puede leer Caso Gürtel. Cuando Resano se refiere a que “la pantalla se ha ido a negro” quiere decir que el vídeo anterior se ha quedado bloqueado con un fondo negro en lugar de emitirse las imágenes que correspondían en ese momento. Desde laSexta nos facilitan el acceso 

In [10]:
from datetime import date

date = date.today().strftime("%d-%m-%Y")

filename = 'Newtral_FN_' + date
print(filename)

outfile = open(filename,'wb')

pickle.dump(lista_articulos, outfile)
outfile.close()

Newtral_FN_28-05-2020


In [11]:
infile = open(filename,'rb')
new_dict = pickle.load(infile)
infile.close()


print(new_dict==lista_articulos)
print(type(new_dict))
new_dict

True
<class 'list'>


[{'Title': 'El video de Helena Resano pidiendo disculpas tras emitirse un fragmento de entrevista con el presidente de SOS Racismo está manipulado',
  'Publication_date': '28 mayo 2020',
  'Content': 'Nos habéis preguntado por este supuesto vídeo de laSexta Noticias en el que se ve una entrevista a Moha Gerehou, presidente de SOS Racismo, que se corta repentinamente y se va a un plano de la periodista Helena Resano en el que dice: “Pedimos disculpas por ese negro que nos ha entrado en ese vídeo en la parte final de esa información”. Varias cuentas de Twitter lo han compartido pero es un FAKE, se trata de un vídeo manipulado.El informativo de Helena Resano es del 9 diciembre de 2015 y se ve en el fondo una pantalla donde se puede leer Caso Gürtel. Cuando Resano se refiere a que “la pantalla se ha ido a negro” quiere decir que el vídeo anterior se ha quedado bloqueado con un fondo negro en lugar de emitirse las imágenes que correspondían en ese momento. Desde laSexta nos facilitan el acc